In [22]:
import requests
import json
from pprint import pprint as ppimport 

In [7]:
site = 'http://store.steampowered.com/api/appdetails?appids=57690'

In [8]:
r = requests.get(site)

In [14]:
the_json  = r.json()


In [25]:
the_json["57690"]['data'][u'price_overview']

{u'currency': u'USD', u'initial': 1999, u'final': 1999, u'discount_percent': 0}

In [29]:
def get_current_price(appid_in):
    site = 'http://store.steampowered.com/api/appdetails?appids=%d' % appid_in
    r = requests.get(site)
    the_json  = r.json()
    
    data = the_json[str(appid_in)]['data'][u'price_overview']
    disc = data['discount_percent']
    ip =  "$%0.2f" % (data['initial'] * .01)
    fp =  "$%0.2f" % (data['final'] * .01)
    discount_percent = "%d%%" % (disc)
    if  disc > 0:
        out_str = 'is currently on sale for %s at %s off at.' % (fp, disc)
    else:
        out_str = 'is currently %s' % fp
    return out_str
    

In [30]:
out_str = get_current_price(57690)

In [31]:
out_str

'is currently $19.99'